In [7]:
import os

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw
from PIL.ImageFilter import FIND_EDGES, SHARPEN
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from face_recognition.api import get_known_people_from_dataset


def add_noise(img):
    VARIABILITY = 40
    deviation = VARIABILITY * np.random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img


def add_objects(img):
    im = Image.fromarray(img.astype('uint8'), 'RGB')
    draw = ImageDraw.Draw(im)

    for i in range(10):
        num1 = np.random.randint(0, 255)
        num2 = np.random.randint(0, 255)
        num3 = np.random.randint(0, 255)
        width1 = np.random.randint(-im.size[0], im.size[0])
        width2 = np.random.randint(-im.size[0], im.size[0])

        draw.line((width1, im.size[1], im.size[0], width2), fill=(num1, num2, num3), width=2)

    return im


def img_enhance(img):
    PIL_image = Image.fromarray(img.astype('uint8'), 'RGB')
    return PIL_image.convert(mode='YCbCr')


def find_img_edges(image):
    PIL_image = Image.fromarray(image.astype('uint8'), 'RGB')
    img1 = PIL_image.filter(FIND_EDGES)
    return img1.filter(SHARPEN)


def generate_augmented(datagen, location, name):
    for subdir, dirs, files in os.walk(f'../dataset/{name}'):
        for idx, file in enumerate(files):
            if idx > 200:
                break
            if file != '.DS_Store' and not file.endswith('-aug.png'):
                filepath = subdir + os.sep + file
                img = load_img(filepath)
                data = img_to_array(img)
                samples = np.expand_dims(data, 0)

                it = datagen.flow(samples, batch_size=1)
                batch = it.next()
                image = batch[0].astype('uint8')

                plt.axis('off')
                plt.imshow(image)
                plt.box(False)
                p = os.path.sep.join([f'../dataset/{name}', f"{str(idx).zfill(5)}-{location}-aug.png"])
                plt.savefig(p, bbox_inches='tight', pad_inches=0)
                plt.close()


names = get_known_people_from_dataset()
for name in names[13:]:
    print(name)
    datagen1 = ImageDataGenerator(brightness_range=[0.3, 0.8])
    generate_augmented(datagen1, 'B', name)
    print('brightness_range')

    datagen2 = ImageDataGenerator(horizontal_flip=True)
    generate_augmented(datagen2, 'horizontal_flip', name)
    print('horizontal_flip')

    datagen3 = ImageDataGenerator(preprocessing_function=add_noise)
    generate_augmented(datagen3, 'random_noise', name)
    print('random_noise')

    datagen4 = ImageDataGenerator(rotation_range=360)
    generate_augmented(datagen4, 'random_rotate', name)
    print('random_rotate')

yt2
brightness_range
horizontal_flip
random_noise
random_rotate
f
brightness_range
horizontal_flip
random_noise
random_rotate
test_person_1


KeyboardInterrupt: 